python 3.6,tensorflow is 1.5.0 and gpflow is 1.1.1

In [2]:
import numpy as np
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import gpflow
# from pyDOE import lhs
from botorch.test_functions import Beale,Branin,SixHumpCamel,Hartmann

from typing import Any
import torch
from botorch.utils.transforms import unnormalize,normalize

In [3]:
class obj_fun:
    def __init__(self,botorch_fun) -> None:
        self.fun = botorch_fun
        self.dim = botorch_fun.dim
        self.bounds = botorch_fun.bounds
    
    def __call__(self, x) -> Any:  # x is a numpy
        x = torch.tensor(x)
        x_unnormal = unnormalize(x, self.bounds).reshape(-1,self.dim)  
        res = self.fun(x_unnormal)

        return np.atleast_2d(res.numpy()).reshape(-1,1)

In [4]:
import torch
from botorch.utils.sampling import draw_sobol_samples


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

def get_initial_points_normal(bounds,num,device,dtype,seed=0):

        #bounds =  torch.tensor([0.,1.]*dim).reshape(-1,2).T
    
        train_x = draw_sobol_samples(
        bounds=bounds, n=num, q=1,seed=seed).reshape(num,-1).to(device, dtype=dtype)
        
        return train_x

In [7]:
sig_e = 1e-4
eps = 1e-4

def K_np(x_data, phi, sigma):
    x_data = np.multiply(x_data, np.reshape(np.exp(phi), [1, -1]))
    dist = np.sum(np.square(x_data), axis=1)
    dist = np.reshape(dist, [-1, 1])
    sq_dists = np.add(np.subtract(dist, 2 * np.matmul(x_data, np.transpose(x_data))), np.transpose(dist))
    my_kernel = np.exp(sigma) * (np.exp(-sq_dists)) + sig_e * np.diag(np.ones(np.shape(x_data)[0]))
    return my_kernel
def K_star_np(x_data, X_M, phi, sigma):
    x_data = np.multiply(x_data, np.reshape(np.exp(phi), [1, -1]))
    X_M = np.multiply(X_M, np.reshape(np.exp(phi), [1, -1]))

    rA = np.reshape(np.sum(np.square(x_data), 1), [-1, 1])
    rB = np.reshape(np.sum(np.square(X_M), 1), [-1, 1])
    pred_sq_dist = np.add(np.subtract(rA, np.multiply(2., np.matmul(x_data, np.transpose(X_M)))), np.transpose(rB))
    pred_kernel = np.exp(sigma) * np.exp(-pred_sq_dist)

    return pred_kernel
def Posterior(x_data, X_M, phi, sigma, y_data, low_bdd, q_mu):
    K_M_0 = np.transpose(K_star_np(x_data, X_M, phi, 0.0))
    K_0 = K_np(x_data, phi, 0.0)
    K_0_inv = np.linalg.inv(K_0)
    GP_mu_M = np.matmul(np.matmul(K_M_0, K_0_inv), y_data)
    ratio_down = (1.0 - np.matmul(np.matmul(K_M_0, K_0_inv), np.transpose(K_M_0)))

    x = np.arange(0, 1, 1.0 / 100.0)
    y = np.arange(0, 1, 1.0 / 100.0)

    X, Y = np.meshgrid(x, y)

    for k in range(X.shape[0]):
        for j in range(Y.shape[0]):
            xx = np.reshape(np.array([X[k, j], Y[k, j]]), [1, 2])
            K_star_0 = np.transpose(K_star_np(x_data, xx, phi, 0.0))
            K_star_M_0 = K_star_np(xx, X_M, phi, 0.0)
            GP_mu_star = np.matmul(np.matmul(K_star_0, K_0_inv), y_data)
            GP_var_star = np.exp(sigma) * (1.0 - np.matmul(np.matmul(K_star_0, K_0_inv), np.transpose(K_star_0)))
            ratio_up = (K_star_M_0 - np.matmul(np.matmul(K_star_0, K_0_inv), np.transpose(K_M_0)))
            GPIO_mean = GP_mu_star + (ratio_up / (ratio_down+eps)) * (q_mu+low_bdd-GP_mu_M)
            GPIO_var = np.maximum(GP_var_star - np.square(ratio_up / (ratio_down+eps)) * (np.exp(sigma) * ratio_down),0.0)
            if k == 0 and j == 0:
                max_coor = xx
                max_value = GPIO_mean + 2.0 * np.sqrt(GPIO_var)
                max_mean=GPIO_mean
                max_var=GPIO_var
            else:
                cur_value = GPIO_mean + 2.0 * np.sqrt(GPIO_var)
                if max_value < cur_value:
                    max_coor = xx
                    max_value = cur_value
                    max_mean=GPIO_mean
                    max_var=GPIO_var
    if np.sum(np.sum(np.abs(x_data-max_coor),axis=1)==0.0) !=0.0:
        max_coor=X_M

    return max_coor,max_mean,max_var
def K(x_data, phi, sigma):
    x_data = tf.multiply(x_data, tf.reshape(tf.exp(phi), [1, -1]))
    dist = tf.reduce_sum(tf.square(x_data), 1)
    dist = tf.reshape(dist, [-1, 1])
    sq_dists = tf.add(tf.subtract(dist, 2 * tf.matmul(x_data, tf.transpose(x_data))), tf.transpose(dist))
    my_kernel = tf.exp(sigma) * (tf.exp(-sq_dists)) + sig_e * tf.reshape(tf.diag(tf.ones([1, tf.shape(x_data)[0]])),
                                                                         [tf.shape(x_data)[0], tf.shape(x_data)[0]])
    return my_kernel
def K_star(x_data, X_M, phi, sigma):
    x_data = tf.multiply(x_data, tf.reshape(tf.exp(phi), [1, -1]))
    X_M = tf.multiply(X_M, tf.reshape(tf.exp(phi), [1, -1]))

    rA = tf.reshape(tf.reduce_sum(tf.square(x_data), 1), [-1, 1])
    rB = tf.reshape(tf.reduce_sum(tf.square(X_M), 1), [-1, 1])
    pred_sq_dist = tf.add(tf.subtract(rA, tf.multiply(2., tf.matmul(x_data, tf.transpose(X_M)))), tf.transpose(rB))
    pred_kernel = tf.exp(sigma) * tf.exp(-pred_sq_dist)
    return pred_kernel
def lbeta(alpha,beta):
    result=tf.lgamma(alpha)+tf.lgamma(beta)-tf.lgamma(alpha+beta)
    return result
def ELBO(x_data,y_data,low_bdd,upp_bdd,X_M,phi,sigma,alpha_q,beta_q,lamda_p):
    eps = 1e-8
    k_vec = K_star(x_data, X_M, phi, 0.0)
    k_vecT = tf.transpose(k_vec)

    R = y_data - (low_bdd) * k_vec
    RT = tf.transpose(R)

    Sig = tf.exp(sigma) * (K(x_data, phi, 0.0) - tf.matmul(k_vec, k_vecT)) + sig_e * tf.reshape(
        tf.diag(tf.ones([1, tf.shape(x_data)[0]])), [tf.shape(x_data)[0], tf.shape(x_data)[0]])
    Sig_inv = tf.matrix_inverse(Sig)

    alpha_pos = tf.exp(alpha_q)
    beta_pos = tf.exp(beta_q)
    lamda_pos = tf.exp(lamda_p)

    KL_div1 = lbeta(1.0,lamda_pos)-lbeta(alpha_pos,beta_pos)+(alpha_pos-1.0)*tf.digamma(alpha_pos)+(beta_pos-lamda_pos)*tf.digamma(beta_pos)
    KL_div2= (1.0+lamda_pos-alpha_pos-beta_pos)*tf.digamma(alpha_pos+beta_pos)

    E1_q = alpha_pos / (alpha_pos + beta_pos)
    E2_q = alpha_pos * beta_pos / (tf.square(alpha_pos + beta_pos) * (alpha_pos + beta_pos + 1)) + tf.square(E1_q)

    KL_div=KL_div1+KL_div2
    Fac1= -0.5 * tf.reduce_sum(tf.log(tf.abs(tf.diag_part(tf.cholesky(Sig))) + eps))- 0.5 * tf.matmul(
        tf.matmul(RT, Sig_inv), R)
    Fac2=E1_q*(upp_bdd-low_bdd)*tf.matmul(tf.matmul(k_vecT,Sig_inv),R)-0.5*E2_q*tf.square(upp_bdd-low_bdd)*tf.matmul(tf.matmul(k_vecT,Sig_inv),k_vec)

    result=Fac1+Fac2-KL_div
    return -result
def acq_fcn_EI(xx,x_data, y_data, low_bdd,upp_bdd, X_M, phi, sigma, q_mu,q_var):
    K_M_0=tf.transpose(K_star(x_data,X_M,phi,0.0))
    K_0 = K(x_data, phi, 0.0)
    K_0_inv = tf.matrix_inverse(K_0)
    GP_mu_M = tf.matmul(tf.matmul(K_M_0, K_0_inv), y_data)
    ratio_down = (1.0 - tf.matmul(tf.matmul(K_M_0, K_0_inv), tf.transpose(K_M_0)))

    K_star_0 = tf.transpose(K_star(x_data, xx, phi, 0.0))
    K_star_M_0 = K_star(xx, X_M, phi, 0.0)
    GP_mu_star = tf.matmul(tf.matmul(K_star_0, K_0_inv), y_data)
    GP_var_star = tf.exp(sigma) * (1.0 - tf.matmul(tf.matmul(K_star_0, K_0_inv), tf.transpose(K_star_0)))
    ratio_up = (K_star_M_0 - tf.matmul(tf.matmul(K_star_0, K_0_inv), tf.transpose(K_M_0)))
    OBCGP_mean = GP_mu_star + (ratio_up / (ratio_down + eps)) * ((upp_bdd - low_bdd) * q_mu + low_bdd - GP_mu_M)
    OBCGP_var = tf.nn.relu(GP_var_star - tf.square(ratio_up / (ratio_down + eps)) * (tf.exp(sigma) * ratio_down))+q_var

    y_max=tf.reduce_max(y_data)
    tau=tf.divide(y_max-OBCGP_mean,tf.sqrt(OBCGP_var)+eps)

    dist = tf.distributions.Normal(loc=0.0,scale=1.0)
    fcn_val= (dist.prob(tau)-tau*dist.cdf(tau))*tf.sqrt(OBCGP_var)

    return fcn_val
def par_update(X, Y,dim):
    with tf.Session() as sess:
        model = gpflow.models.GPR(X, Y, gpflow.kernels.RBF(dim, ARD=False))
        model.clear()
        model.likelihood.variance = sig_e
    
        model.kern.lengthscales.prior = gpflow.priors.Gamma(1, 1)
        model.kern.variance.prior = gpflow.priors.Gamma(1., 1.)
    
        model.compile()
        opt = gpflow.train.ScipyOptimizer()
        opt.minimize(model)

    sigma_np = np.float32(np.log(model.kern.variance.value))
    phi_np = np.reshape(np.float32(0.5 * (-np.log(2.0) - 2.0 * np.log(model.kern.lengthscales.value))), [-1, 1])

    return sigma_np, phi_np

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double


function_information = []



temp={}
temp['name']='Branin2D' 
temp['function'] = Branin(negate=True)
temp['fstar'] =  -0.397887 
function_information.append(temp)

# temp={}
# temp['name']='SixHumpCamel2D' 
# temp['function'] = SixHumpCamel(negate=True)
# temp['fstar'] =  1.0317
# function_information.append(temp)

# temp={}
# temp['name']='Hartmann3D' 
# temp['function'] = Hartmann(dim=3,negate=True)
# temp['fstar'] =  3.86278
# function_information.append(temp)



for information in function_information:

    fun = information['function']
    fstar = information['fstar']
    print('fstar is: ',fstar)

   
    neg_fun = obj_fun(fun)

    dim=neg_fun.dim
    bounds=neg_fun.bounds
    num_ini=4*dim
    priori_known_mode=True
    eps = 1e-4
    Max_iter = 30
    lr = 1e-1
    sig_e = 1e-4

    upp_bdd_prior = fstar

    step_size = 3

    OBCGP = []

    N = 2

    for exp in range(N):


        X = get_initial_points_normal(bounds=bounds,num=num_ini,device=device,dtype=dtype,seed=exp)
        X = normalize(X, bounds)
        X = X.numpy()
        Y=neg_fun(X)
        Y_best=np.max(Y)
        Y_rescale=Y-Y_best

        low_bdd_feed=0.0
        upp_bdd_feed=upp_bdd_prior-Y_best

        lamda_feed = np.float32(np.log(10.0))
        alpha_feed = np.float32(0.0)
        beta_feed = lamda_feed
        X_M_t = np.reshape(X[np.argmax(Y)], (1, dim))
        X_M_t_feed = np.log(np.divide(X_M_t, 1.0 - X_M_t + eps)) + np.random.normal(0.0, 0.01, [1, dim])

        best_record = []
        best_record.append(-np.max(Y))
        print('initial best is: ',best_record[-1])

        for iter in range(100):

            tf.set_random_seed(1234)
            np.random.seed(1234) 

            try:
                if iter%step_size == 0:
                    sigma_feed, phi_feed = par_update(X, Y_rescale,dim)

                tf.reset_default_graph()
                alpha_v = tf.Variable(tf.convert_to_tensor(alpha_feed, dtype=tf.float32))
                beta_v = tf.Variable(tf.convert_to_tensor(beta_feed, dtype=tf.float32))
                X_M_t_v = tf.Variable(tf.convert_to_tensor(X_M_t_feed, tf.float32))
                X_M = (tf.sigmoid(X_M_t_v))
                x_data_p = tf.convert_to_tensor(X, dtype=tf.float32)
                y_data_p = tf.convert_to_tensor(Y_rescale, dtype=tf.float32)
                low_bdd_p = tf.convert_to_tensor(low_bdd_feed, tf.float32)
                upp_bdd_p=tf.convert_to_tensor(upp_bdd_feed,tf.float32)
                lamda_p = tf.convert_to_tensor(lamda_feed, dtype=tf.float32)
                phi_p = (tf.convert_to_tensor(phi_feed, dtype=tf.float32))
                sigma_p = (tf.convert_to_tensor(sigma_feed, dtype=tf.float32))

                pt_sel = tf.Variable(tf.convert_to_tensor(X_M_t_feed, tf.float32))
                costF = ELBO(x_data_p, y_data_p, low_bdd_p,upp_bdd_p, X_M, phi_p, sigma_p, alpha_v, beta_v, lamda_p)
                q_mu_p=tf.exp(alpha_v) / (tf.exp(alpha_v) + tf.exp(beta_v))
                q_var_p=tf.divide(tf.exp(alpha_v+beta_v),tf.square(tf.exp(alpha_v)+tf.exp(beta_v))*(1.0+tf.exp(alpha_v)+tf.exp(beta_v)))
                acq_fcn_val = acq_fcn_EI(tf.sigmoid(pt_sel), x_data_p, y_data_p, low_bdd_p,upp_bdd_p, X_M, phi_p, sigma_p,q_mu_p,q_var_p)

                optimizer = tf.train.AdamOptimizer(lr)
            
                train_par = optimizer.minimize(costF)
                train_acq = optimizer.minimize(-acq_fcn_val, var_list=pt_sel)

                sess = tf.Session()
                init = tf.global_variables_initializer()
                sess.run(init)

                for opt_iter in range(Max_iter):
                    sess.run(train_par)
                for opt_iter_acq in range(Max_iter):
                    sess.run(train_acq)
                X_M_np, phi_np, sigma_np, alpha_np, beta_np = sess.run((X_M, phi_p, sigma_p, alpha_v, beta_v))
                q_mu = np.exp(alpha_np - beta_np)
                q_var = np.exp(alpha_np - 2.0 * beta_np)

                pt_next=sess.run(tf.sigmoid(pt_sel))
                val_next = np.reshape(neg_fun(pt_next), [])

                X = np.concatenate((X, pt_next), axis=0)
                Y = np.concatenate((Y, np.reshape(val_next, [1, 1])), axis=0)

                if val_next > Y_best:
                    X_M_t_feed = np.log(np.divide(pt_next + eps, 1.0 - pt_next + eps)) + np.random.normal(0.0, 0.01, [1, dim])
                    alpha_feed = np.float32(0.0)
                    beta_feed = lamda_feed
                    X_M_t = pt_next
                    Y_best = val_next
                    low_bdd_feed = 0.0
                    upp_bdd_feed= upp_bdd_prior-val_next
                    Y_rescale = Y - Y_best
                else:
                    X_M_t_feed = np.log(np.divide(X_M_t + eps, 1.0 - X_M_t + eps)) + np.random.normal(0.0, 0.01, [1, dim])
                    alpha_feed = np.float32(0.0)
                    beta_feed = lamda_feed
                    Y_rescale = Y - Y_best

                sess.close()
                tf.reset_default_graph()
                X_M_best = np.reshape(X[np.argmax(Y)], (1, dim))

                best_record.append( -np.max(Y))
            except:
                print('oh no!!')
                best_record.append( -np.max(Y))
            
            print('iteration #:%d\t' % (iter + 1) + 'current Minimum value: %f\t' % (-np.max(Y)) )

        OBCGP.append(best_record)

fstar is:  -0.397887
initial best is:  3.5451968551073154


c:\Users\Hanyang.Wang\anaconda3\envs\tf1\lib\site-packages\gpflow\densities.py:89: UserWarning: Shape of x must be 2D at computation.
  warnings.warn('Shape of x must be 2D at computation.')


oh no!!
iteration #:1	current Minimum value: 3.545197	
oh no!!
iteration #:2	current Minimum value: 3.545197	
oh no!!
iteration #:3	current Minimum value: 3.545197	
oh no!!
iteration #:4	current Minimum value: 3.545197	
oh no!!
iteration #:5	current Minimum value: 3.545197	
oh no!!
iteration #:6	current Minimum value: 3.545197	
oh no!!
iteration #:7	current Minimum value: 3.545197	
oh no!!
iteration #:8	current Minimum value: 3.545197	
oh no!!
iteration #:9	current Minimum value: 3.545197	
oh no!!
iteration #:10	current Minimum value: 3.545197	
oh no!!
iteration #:11	current Minimum value: 3.545197	
oh no!!
iteration #:12	current Minimum value: 3.545197	
oh no!!
iteration #:13	current Minimum value: 3.545197	
oh no!!
iteration #:14	current Minimum value: 3.545197	
oh no!!
iteration #:15	current Minimum value: 3.545197	
oh no!!
iteration #:16	current Minimum value: 3.545197	
oh no!!
iteration #:17	current Minimum value: 3.545197	
oh no!!
iteration #:18	current Minimum value: 3.545197	
o

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double


function_information = []



temp={}
temp['name']='Branin2D' 
temp['function'] = Branin(negate=True)
temp['fstar'] =  -0.397887 
function_information.append(temp)

# temp={}
# temp['name']='SixHumpCamel2D' 
# temp['function'] = SixHumpCamel(negate=True)
# temp['fstar'] =  1.0317
# function_information.append(temp)

# temp={}
# temp['name']='Hartmann3D' 
# temp['function'] = Hartmann(dim=3,negate=True)
# temp['fstar'] =  3.86278
# function_information.append(temp)



for information in function_information:

    fun = information['function']
    fstar = information['fstar']
    print('fstar is: ',fstar)

   
    neg_fun = obj_fun(fun)

    dim=neg_fun.dim
    bounds=neg_fun.bounds
    num_ini=4*dim
    priori_known_mode=True
    eps = 1e-4
    Max_iter = 30
    lr = 1e-1
    sig_e = 1e-4

    upp_bdd_prior = fstar

    step_size = 3

    OBCGP = []

    N = 2

    for exp in range(N):


        X = get_initial_points_normal(bounds=bounds,num=num_ini,device=device,dtype=dtype,seed=exp)
        X = normalize(X, bounds)
        X = X.numpy()
        Y=neg_fun(X)
        Y_best=np.max(Y)
        Y_rescale=Y-Y_best

        low_bdd_feed=0.0
        upp_bdd_feed=upp_bdd_prior-Y_best

        lamda_feed = np.float32(np.log(10.0))
        alpha_feed = np.float32(0.0)
        beta_feed = lamda_feed
        X_M_t = np.reshape(X[np.argmax(Y)], (1, dim))
        X_M_t_feed = np.log(np.divide(X_M_t, 1.0 - X_M_t + eps)) + np.random.normal(0.0, 0.01, [1, dim])

        best_record = []
        best_record.append(-np.max(Y))
        print('initial best is: ',best_record[-1])

        for iter in range(100):


            tf.set_random_seed(1234)
            np.random.seed(1234) 

            try:
                if iter%step_size == 0:
                    sigma_feed, phi_feed = par_update(X, Y_rescale,dim)

                tf.reset_default_graph()
                alpha_v = tf.Variable(tf.convert_to_tensor(alpha_feed, dtype=tf.float32))
                beta_v = tf.Variable(tf.convert_to_tensor(beta_feed, dtype=tf.float32))
                X_M_t_v = tf.Variable(tf.convert_to_tensor(X_M_t_feed, tf.float32))
                X_M = (tf.sigmoid(X_M_t_v))
                x_data_p = tf.convert_to_tensor(X, dtype=tf.float32)
                y_data_p = tf.convert_to_tensor(Y_rescale, dtype=tf.float32)
                low_bdd_p = tf.convert_to_tensor(low_bdd_feed, tf.float32)
                upp_bdd_p=tf.convert_to_tensor(upp_bdd_feed,tf.float32)
                lamda_p = tf.convert_to_tensor(lamda_feed, dtype=tf.float32)
                phi_p = (tf.convert_to_tensor(phi_feed, dtype=tf.float32))
                sigma_p = (tf.convert_to_tensor(sigma_feed, dtype=tf.float32))

                pt_sel = tf.Variable(tf.convert_to_tensor(X_M_t_feed, tf.float32))
                costF = ELBO(x_data_p, y_data_p, low_bdd_p,upp_bdd_p, X_M, phi_p, sigma_p, alpha_v, beta_v, lamda_p)
                q_mu_p=tf.exp(alpha_v) / (tf.exp(alpha_v) + tf.exp(beta_v))
                q_var_p=tf.divide(tf.exp(alpha_v+beta_v),tf.square(tf.exp(alpha_v)+tf.exp(beta_v))*(1.0+tf.exp(alpha_v)+tf.exp(beta_v)))
                acq_fcn_val = acq_fcn_EI(tf.sigmoid(pt_sel), x_data_p, y_data_p, low_bdd_p,upp_bdd_p, X_M, phi_p, sigma_p,q_mu_p,q_var_p)

                optimizer = tf.train.AdamOptimizer(lr)
            
                train_par = optimizer.minimize(costF)
                train_acq = optimizer.minimize(-acq_fcn_val, var_list=pt_sel)

                sess = tf.Session()
                init = tf.global_variables_initializer()
                sess.run(init)

                for opt_iter in range(Max_iter):
                    sess.run(train_par)
                for opt_iter_acq in range(Max_iter):
                    sess.run(train_acq)
                X_M_np, phi_np, sigma_np, alpha_np, beta_np = sess.run((X_M, phi_p, sigma_p, alpha_v, beta_v))
                q_mu = np.exp(alpha_np - beta_np)
                q_var = np.exp(alpha_np - 2.0 * beta_np)

                pt_next=sess.run(tf.sigmoid(pt_sel))
                val_next = np.reshape(neg_fun(pt_next), [])

                X = np.concatenate((X, pt_next), axis=0)
                Y = np.concatenate((Y, np.reshape(val_next, [1, 1])), axis=0)

                if val_next > Y_best:
                    X_M_t_feed = np.log(np.divide(pt_next + eps, 1.0 - pt_next + eps)) + np.random.normal(0.0, 0.01, [1, dim])
                    alpha_feed = np.float32(0.0)
                    beta_feed = lamda_feed
                    X_M_t = pt_next
                    Y_best = val_next
                    low_bdd_feed = 0.0
                    upp_bdd_feed= upp_bdd_prior-val_next
                    Y_rescale = Y - Y_best
                else:
                    X_M_t_feed = np.log(np.divide(X_M_t + eps, 1.0 - X_M_t + eps)) + np.random.normal(0.0, 0.01, [1, dim])
                    alpha_feed = np.float32(0.0)
                    beta_feed = lamda_feed
                    Y_rescale = Y - Y_best

                sess.close()
                tf.reset_default_graph()
                X_M_best = np.reshape(X[np.argmax(Y)], (1, dim))

                best_record.append( -np.max(Y))
            except:
                print('oh no!!')
                best_record.append( -np.max(Y))
            
            print('iteration #:%d\t' % (iter + 1) + 'current Minimum value: %f\t' % (-np.max(Y)) )

        OBCGP.append(best_record)

fstar is:  -0.397887
initial best is:  3.5451968551073154


c:\Users\Hanyang.Wang\anaconda3\envs\tf1\lib\site-packages\gpflow\densities.py:89: UserWarning: Shape of x must be 2D at computation.
  warnings.warn('Shape of x must be 2D at computation.')


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 231.903903
  Number of iterations: 29
  Number of functions evaluations: 30
iteration #:1	current Minimum value: 3.545197	
iteration #:2	current Minimum value: 2.223923	
iteration #:3	current Minimum value: 2.223923	
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 244.283299
  Number of iterations: 31
  Number of functions evaluations: 33
iteration #:4	current Minimum value: 1.098476	
iteration #:5	current Minimum value: 1.098476	
iteration #:6	current Minimum value: 1.098476	
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 77.141865
  Number of iterations: 51
  Number of functions evaluations: 60
iteration #:7	current Minimum value: 1.098476	
iteration #:8	current Minimum value

In [10]:
get_initial_points_normal(bounds=bounds,num=num_ini,device=device,dtype=dtype,seed=1)

tensor([[-0.9841,  5.4263],
        [ 7.6516, 14.8771],
        [ 4.7489,  1.9125],
        [-1.8937,  7.6096],
        [-3.2310,  1.7822],
        [ 3.2806, 11.1965],
        [ 9.8152,  5.7611],
        [ 1.5466, 11.4364]], dtype=torch.float64)

In [7]:
#neg_fun = obj_fun(Branin(negate=True))
neg_fun = obj_fun(Hartmann(dim=3,negate=True))

dim=neg_fun.dim
bounds=neg_fun.bounds
num_ini=4*dim
priori_known_mode=True
eps = 1e-4
Max_iter = 30
lr = 1e-1
sig_e = 1e-4

upp_bdd_prior=3.86278  #-0.397887  #1.0

step_size = 3


X = get_initial_points_normal(bounds=bounds,num=num_ini,device=device,dtype=dtype,seed=0)
X = normalize(X, bounds)
X = X.numpy()
Y=neg_fun(X)
Y_best=np.max(Y)
Y_rescale=Y-Y_best

low_bdd_feed=0.0
upp_bdd_feed=upp_bdd_prior-Y_best

lamda_feed = np.float32(np.log(10.0))
alpha_feed = np.float32(0.0)
beta_feed = lamda_feed
X_M_t = np.reshape(X[np.argmax(Y)], (1, dim))
X_M_t_feed = np.log(np.divide(X_M_t, 1.0 - X_M_t + eps)) + np.random.normal(0.0, 0.01, [1, dim])

best_record = []

for iter in range(100):

    tf.set_random_seed(1)

    try:
        if iter%step_size == 0:
            sigma_feed, phi_feed = par_update(X, Y_rescale,dim)

        tf.reset_default_graph()
        alpha_v = tf.Variable(tf.convert_to_tensor(alpha_feed, dtype=tf.float32))
        beta_v = tf.Variable(tf.convert_to_tensor(beta_feed, dtype=tf.float32))
        X_M_t_v = tf.Variable(tf.convert_to_tensor(X_M_t_feed, tf.float32))
        X_M = (tf.sigmoid(X_M_t_v))
        x_data_p = tf.convert_to_tensor(X, dtype=tf.float32)
        y_data_p = tf.convert_to_tensor(Y_rescale, dtype=tf.float32)
        low_bdd_p = tf.convert_to_tensor(low_bdd_feed, tf.float32)
        upp_bdd_p=tf.convert_to_tensor(upp_bdd_feed,tf.float32)
        lamda_p = tf.convert_to_tensor(lamda_feed, dtype=tf.float32)
        phi_p = (tf.convert_to_tensor(phi_feed, dtype=tf.float32))
        sigma_p = (tf.convert_to_tensor(sigma_feed, dtype=tf.float32))

        pt_sel = tf.Variable(tf.convert_to_tensor(X_M_t_feed, tf.float32))
        costF = ELBO(x_data_p, y_data_p, low_bdd_p,upp_bdd_p, X_M, phi_p, sigma_p, alpha_v, beta_v, lamda_p)
        q_mu_p=tf.exp(alpha_v) / (tf.exp(alpha_v) + tf.exp(beta_v))
        q_var_p=tf.divide(tf.exp(alpha_v+beta_v),tf.square(tf.exp(alpha_v)+tf.exp(beta_v))*(1.0+tf.exp(alpha_v)+tf.exp(beta_v)))
        acq_fcn_val = acq_fcn_EI(tf.sigmoid(pt_sel), x_data_p, y_data_p, low_bdd_p,upp_bdd_p, X_M, phi_p, sigma_p,q_mu_p,q_var_p)

        optimizer = tf.train.AdamOptimizer(lr)
      
        train_par = optimizer.minimize(costF)
        train_acq = optimizer.minimize(-acq_fcn_val, var_list=pt_sel)

        sess = tf.Session()
        init = tf.global_variables_initializer()
        sess.run(init)

        for opt_iter in range(Max_iter):
            sess.run(train_par)
        for opt_iter_acq in range(Max_iter):
            sess.run(train_acq)
        X_M_np, phi_np, sigma_np, alpha_np, beta_np = sess.run((X_M, phi_p, sigma_p, alpha_v, beta_v))
        q_mu = np.exp(alpha_np - beta_np)
        q_var = np.exp(alpha_np - 2.0 * beta_np)

        pt_next=sess.run(tf.sigmoid(pt_sel))
        val_next = np.reshape(neg_fun(pt_next), [])

        X = np.concatenate((X, pt_next), axis=0)
        Y = np.concatenate((Y, np.reshape(val_next, [1, 1])), axis=0)

        if val_next > Y_best:
            X_M_t_feed = np.log(np.divide(pt_next + eps, 1.0 - pt_next + eps)) + np.random.normal(0.0, 0.01, [1, dim])
            alpha_feed = np.float32(0.0)
            beta_feed = lamda_feed
            X_M_t = pt_next
            Y_best = val_next
            low_bdd_feed = 0.0
            upp_bdd_feed= upp_bdd_prior-val_next
            Y_rescale = Y - Y_best
        else:
            X_M_t_feed = np.log(np.divide(X_M_t + eps, 1.0 - X_M_t + eps)) + np.random.normal(0.0, 0.01, [1, dim])
            alpha_feed = np.float32(0.0)
            beta_feed = lamda_feed
            Y_rescale = Y - Y_best

        sess.close()
        tf.reset_default_graph()
        X_M_best = np.reshape(X[np.argmax(Y)], (1, dim))

        best_record.append( -np.max(Y))
    except:
        print('oh shit!!')
        best_record.append( -np.max(Y))
    
    print('iteration #:%d\t' % (iter + 1) + 'current Minimum value: %f\t' % (-np.max(Y)) )

c:\Users\Hanyang.Wang\anaconda3\envs\tf1\lib\site-packages\gpflow\densities.py:89: UserWarning: Shape of x must be 2D at computation.
  warnings.warn('Shape of x must be 2D at computation.')


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 21.995620
  Number of iterations: 14
  Number of functions evaluations: 15
iteration #:1	current Minimum value: -2.712463		current Minimum point: (0.793214,0.550066)
iteration #:2	current Minimum value: -2.712463		current Minimum point: (0.793214,0.550066)
iteration #:3	current Minimum value: -2.712463		current Minimum point: (0.793214,0.550066)
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 24.997011
  Number of iterations: 37
  Number of functions evaluations: 43
iteration #:4	current Minimum value: -2.712463		current Minimum point: (0.793214,0.550066)
iteration #:5	current Minimum value: -2.713811		current Minimum point: (0.792650,0.548833)
iteration #:6	current Minimum value: -2.713811		current Minimum point: (0.792650,0.548833)
INFO:tensorflow:Optimization te

In [10]:
model = gpflow.models.GPR(X, Y_rescale, gpflow.kernels.RBF(dim))
print(model)

                             class prior transform  trainable shape  \
GPR/kern/variance        Parameter  None       +ve       True    ()   
GPR/kern/lengthscales    Parameter  None       +ve       True    ()   
GPR/likelihood/variance  Parameter  None       +ve       True    ()   

                         fixed_shape value  
GPR/kern/variance               True   1.0  
GPR/kern/lengthscales           True   1.0  
GPR/likelihood/variance         True   1.0  


c:\Users\Hanyang.Wang\anaconda3\envs\tf1\lib\site-packages\gpflow\densities.py:89: UserWarning: Shape of x must be 2D at computation.
  warnings.warn('Shape of x must be 2D at computation.')
